##LDA

In [ ]:
dictionary = 

###这一步是最初始的文本数据导入 

In [2]:
import pandas as pd     # 数据表
import numpy as np     # 数组
import re     # 正则表达式
import jieba     # 中文分词
import matplotlib.pyplot as plt     # 画图
from gensim import corpora, models
import pyLDAvis     # 交互式LDA可视化
import pyLDAvis.gensim_models as gensimvis

In [4]:
df = pd.read_excel('text_analysis_weibo_sample.xlsx', index_col = 0)
df.head()

,index,标题/微博内容,点赞,转发,评论,账号昵称UID加密,粉丝数,关注数,地域
0,34121,国债：地产行业重磅利好提振风险偏好，期债低开低走 国债期货全线收跌，10年期主力...,0,0,0,e5df796860e68f403bcf9651bab4d42e,0,0,其他
1,40230,#喜迎二十大 忠诚保平安#,0,0,0,6e35cb69ad52f20de5e28197b2e85306,405444,252,广西
2,7714,注意！事关明日教资考试！福建省教育考试院发布补充公告 福建省2022年下半年全国中小学教师...,0,0,0,e6953217442e6c06a7af23eee5e185f2,53264,2177,福建
3,27378,近日，“千年大计”雄安新区迎来五周岁生日。从“一张白纸...,0,0,0,NaN,0,0,北京
4,15435,樊振东牛逼！,0,0,0,344af41eac516375c04dee6325e763cc,8,51,山东


###文本导入成功，进行主题模型处理之前，还有一个必备的步骤：预处理，把不必要的数据给剔除掉，这样才能提高模型的精准率

In [5]:
def remove_nums(text):
    nonums = re.sub('[^\u4e00-\u9fa5]+', '', text)
    return nonums
test = df['标题/微博内容'][0]
remove_nums(test)[:100]

'国债地产行业重磅利好提振风险偏好期债低开低走国债期货全线收跌年期主力合约跌年期主力合约跌年期主力合约跌三大主力合约均创逾一年收盘新低行情解读公开市场方面央行开展了亿元天期逆回购操作中标利率亿元逆回购到'

In [6]:
def clean_text(text):
    words = jieba.lcut(text)
    # words = [w for w in words if w not in stopwords and w!='[' and w!=']']
    return ' '.join(words)

test = df['标题/微博内容'][0]
clean_text(test)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\20693\AppData\Local\Temp\jieba.cache
Loading model cost 1.378 seconds.
Prefix dict has been built successfully.


'    国债 ： 地产 行业 重磅 利好 提振 风险 偏好 ， 期债 低开 低 走           国债 期货 全线 收跌 ， 10 年期 主力 合约 跌 0.37% ， 5 年期 主力 合约 跌 0.24% ， 2 年期 主力 合约 跌 0.11% ， 三大 主力 合约 均 创 逾 一年 收盘 新低 。         行情 解读 ：         1 ， 公开市场 方面 ， 央行 开展 了 800 亿元 7 天期 逆 回购 操作 ， 中标 利率 2.0% ， 20 亿元 逆 回购 到期 ， 因此 当日 净 投放 780 亿元 。         2 ， 资金面 方面 ， 央行 公开市场 逆 回购 继续 加力 ， 不过 银行 间 市场 周二 资金 供给 趋于 收敛 ， 七天 回购 加权 利率 进一步 上行 。 长期 资金 方面 ， 全国 和 主要 股份制 银行 一年期 同业 存单 报价 仍 维持 在 2.45% 水平 ， 不过 目前 尚无 成交量 配合 。         3 ， 房地产 行业 再度 迎来 重磅 利好 。 证监会 宣布 在 涉房 企业 股权 融资 方面 调整 优化 五项 措施 ， 并 自即日起 施行 。 随着 证监会 支持 房企 股权 融资 这 “ 第三支 箭 ” 正式 发射 ， 信贷 、 债券 、 股权 三个 融资 渠道 “ 三箭 齐发 ” ， 合力 推动 房地产 融资 。 — — “ 第三支 箭 ” 快速 落地 ， 或 将 为 房地产 行业 注入 较 大规模 的 增量 资金 和 资产 ， 促进 行业 平稳 健康 发展 。 在 短短 20 天 的 时间 内 政策 端 “ 三箭 齐发 ” 强势 托底 ， 提振 各方 的 信心 。 看好 全 行业 的 信用 复苏 以及 在 因城 施策 空间 逐步 打开 下 的 销售 回暖 。         4 ， 国务院 联防 联控 机制 11 月 29 日 15 : 00 召开 新闻 发布会 。 针对 “ 考虑 到 最近 个别 地方 的 不满情绪 ， 政府 是否 会 重新考虑 疫情 应对 政策 ” 的 问题 ， 国家 卫生 健康 委 新闻 发言人 、 宣传司 副司长 米锋 称 ， 疫情 发生 以来 ， 中国政府 一直 根据 病毒 变异 的 特点 和 临床 治疗 的 实践 认识 ， 边 防控 、 边 研究 

In [7]:
df['标题/微博内容'] = df['标题/微博内容'].astype(str)
df['微博内容分词'] = df['标题/微博内容'].apply(remove_nums)
df['微博内容分词'] = df['微博内容分词'].apply(clean_text)
df['微博内容分词'] = df['微博内容分词'].apply(lambda x: x.split())
df

,index,标题/微博内容,点赞,转发,评论,账号昵称UID加密,粉丝数,关注数,地域,微博内容分词
0,34121,国债：地产行业重磅利好提振风险偏好，期债低开低走 国债期货全线收跌，10年期主力...,0,0,0,e5df796860e68f403bcf9651bab4d42e,0,0,其他,"[国债, 地产, 行业, 重磅, 利好, 提振, 风险, 偏好, 期债, 低开, 低, 走,..."
1,40230,#喜迎二十大 忠诚保平安#,0,0,0,6e35cb69ad52f20de5e28197b2e85306,405444,252,广西,"[喜迎, 二十大, 忠诚, 保平安]"
2,7714,注意！事关明日教资考试！福建省教育考试院发布补充公告 福建省2022年下半年全国中小学教师...,0,0,0,e6953217442e6c06a7af23eee5e185f2,53264,2177,福建,"[注意, 事关, 明日, 教资, 考试, 福建省, 教育, 考试院, 发布, 补充, 公告,..."
3,27378,近日，“千年大计”雄安新区迎来五周岁生日。从“一张白纸...,0,0,0,NaN,0,0,北京,"[近日, 千年, 大计, 雄安, 新区, 迎来, 五周岁, 生日, 从, 一张白纸, 到, ..."
4,15435,樊振东牛逼！,0,0,0,344af41eac516375c04dee6325e763cc,8,51,山东,"[樊振东, 牛, 逼]"
...,...,...,...,...,...,...,...,...,...,...
95,14034,老公好漂亮[舔屏],0,0,0,aa7af97ae98cbe8983e559b948ecfabf,825,191,北京,"[老公, 好, 漂亮, 舔, 屏]"
96,35249,恭喜@张小娜呀娜 1名用户获得【小飞T】。C官方唯一抽奖工具@C抽奖平台 对本次抽奖进行监督...,0,0,2,f538513e5801c275cbcf285517a8ee62,51709,20,北京,"[恭喜, 张小娜, 呀, 娜, 名, 用户, 获得, 小飞, 官方, 唯一, 抽奖, 工具,..."
97,39598,接不动了，居民部门负债率提升空间太有限了。现在动辄几万一平米，总价高，月供高，预期转差，还怎...,0,0,0,2e76c39b6665a0b6f5bdc4c35f36f573,0,59,四川,"[接不动, 了, 居民, 部门, 负债率, 提升, 空间, 太, 有限, 了, 现在, 动辄..."
98,48977,目前电子烟政策已从制定阶段逐渐走向实施阶段，而主要的电子烟公司股价下跌幅度高达70%-...,0,0,0,NaN,0,0,北京,"[目前, 电子, 烟, 政策, 已, 从, 制定, 阶段, 逐渐, 走向, 实施, 阶段, ..."


###接下来是分词的LDA部分

In [9]:

dictionary = corpora.Dictionary(df['微博内容分词'])     # 根据分词结果创建字典
corpus = [dictionary.doc2bow(text) for text in df['微博内容分词']]     # 根据分词结果创建语料库
# 训练LDA模型
lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

In [10]:
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.040*"的" + 0.011*"电子" + 0.011*"烟" + 0.009*"了" + 0.009*"在"')
(1, '0.014*"的" + 0.014*"考生" + 0.009*"考试" + 0.007*"考场" + 0.006*"考点"')
(2, '0.036*"的" + 0.010*"在" + 0.007*"郑州" + 0.007*"月" + 0.005*"和"')
(3, '0.015*"的" + 0.009*"数字" + 0.007*"建设" + 0.006*"年" + 0.005*"城市"')
(4, '0.017*"的" + 0.015*"月" + 0.013*"万吨" + 0.009*"市场" + 0.008*"日"')


In [11]:
df['微博内容分词'].iloc[0]

['国债',
 '地产',
 '行业',
 '重磅',
 '利好',
 '提振',
 '风险',
 '偏好',
 '期债',
 '低开',
 '低',
 '走',
 '国债',
 '期货',
 '全线',
 '收跌',
 '年期',
 '主力',
 '合约',
 '跌',
 '年期',
 '主力',
 '合约',
 '跌',
 '年期',
 '主力',
 '合约',
 '跌三大',
 '主力',
 '合约',
 '均',
 '创',
 '逾',
 '一年',
 '收盘',
 '新低',
 '行情',
 '解读',
 '公开市场',
 '方面',
 '央行',
 '开展',
 '了',
 '亿元',
 '天期',
 '逆',
 '回购',
 '操作',
 '中标',
 '利率',
 '亿元',
 '逆',
 '回购',
 '到期',
 '因此',
 '当日',
 '净',
 '投放',
 '亿元',
 '资金面',
 '方面',
 '央行',
 '公开市场',
 '逆',
 '回购',
 '继续',
 '加力',
 '不过',
 '银行',
 '间',
 '市场',
 '周二',
 '资金',
 '供给',
 '趋于',
 '收敛',
 '七天',
 '回购',
 '加权',
 '利率',
 '进一步',
 '上行',
 '长期',
 '资金',
 '方面',
 '全国',
 '和',
 '主要',
 '股份制',
 '银行',
 '一年期',
 '同业',
 '存单',
 '报价',
 '仍',
 '维持',
 '在',
 '水平',
 '不过',
 '目前',
 '尚无',
 '成交量',
 '配合',
 '房地产',
 '行业',
 '再度',
 '迎来',
 '重磅',
 '利好',
 '证监会',
 '宣布',
 '在',
 '涉房',
 '企业',
 '股权',
 '融资',
 '方面',
 '调整',
 '优化',
 '五项',
 '措施',
 '并',
 '自即日起',
 '施行',
 '随着',
 '证监会',
 '支持',
 '房企',
 '股权',
 '融资',
 '这',
 '第三支',
 '箭',
 '正式',
 '发射',
 '信贷',
 '债券',
 '股权',
 '三个',
 '融资',
 '渠道',
 '三箭',
 '齐发',
 '合

In [12]:
for index, score in sorted(lda_model[corpus[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.9916677474975586	 
Topic: 0.036*"的" + 0.010*"在" + 0.007*"郑州" + 0.007*"月" + 0.005*"和" + 0.005*"是" + 0.005*"了" + 0.005*"亿元" + 0.004*"也" + 0.004*"为"


In [13]:
documents = df['微博内容分词'].values
def infer_topics(lda_model, document):
    bow = dictionary.doc2bow(document)
    topics = lda_model.get_document_topics(bow)
    return topics

In [14]:
for i, doc in enumerate(documents[:10]):
    doc_topics = infer_topics(lda_model, doc)
    print(f"Document {i+1}:")
    print(doc_topics)
    print()

Document 1:
[(2, 0.99156904)]

Document 2:
[(0, 0.83976775), (1, 0.040062875), (2, 0.040051926), (3, 0.04007007), (4, 0.040047318)]

Document 3:
[(1, 0.99739015)]

Document 4:
[(0, 0.99933434)]

Document 5:
[(0, 0.050136838), (1, 0.050170243), (2, 0.050411332), (3, 0.3106986), (4, 0.538583)]

Document 6:
[(0, 0.022373509), (1, 0.022232298), (2, 0.02273805), (3, 0.90941566), (4, 0.023240447)]

Document 7:
[(0, 0.0125769945), (1, 0.012549953), (2, 0.012615908), (3, 0.949685), (4, 0.0125721395)]

Document 8:
[(0, 0.100231834), (1, 0.59569246), (2, 0.10057452), (3, 0.10047958), (4, 0.10302162)]

Document 9:
[(3, 0.9720066)]

Document 10:
[(2, 0.9993377)]



###文本数据分析的最后一步，往往就是可视化

In [15]:
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary, n_jobs=1) 
# 显示可视化界面
pyLDAvis.display(lda_vis)